<a href="https://colab.research.google.com/github/Discovery19/Telecom/blob/main/lab_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Лабораторная №7


In [ ]:
import numpy as np
PI2 = 2 * np.pi

## Задание 1



В качестве тестового примера я начну с небольшого действительного сигнала и вычислю его БПФ:

In [ ]:
ys = [-0.5, 0.1, 0.7, -0.1]
hs = np.fft.fft(ys)
print(hs)

[ 0.2+0.j  -1.2-0.2j  0.2+0.j  -1.2+0.2j]


Вот моя реализация ДПФ из книги:

In [ ]:
def dft(ys):
    N = len(ys)
    ts = np.arange(N) / N
    freqs = np.arange(N)
    args = np.outer(ts, freqs)
    M = np.exp(1j * PI2 * args)
    amps = M.conj().transpose().dot(ys)
    return amps

Мы можем подтвердить, что эта реализация дает тот же результат.

In [ ]:
hs2 = dft(ys)
np.sum(np.abs(hs - hs2))

5.864775846765962e-16

В качестве шага к созданию рекурсивного БПФ я начну с версии, которая разделяет входной массив и использует np.fft.fft для вычисления БПФ половин.

In [ ]:
def fft_norec(ys):
    N = len(ys)
    He = np.fft.fft(ys[::2])
    Ho = np.fft.fft(ys[1::2])

    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)

    return np.tile(He, 2) + W * np.tile(Ho, 2)

И мы получаем те же результаты:

In [ ]:
hs3 = fft_norec(ys)
np.sum(np.abs(hs - hs3))

0.0

Наконец, мы можем заменить `np.fft.fft` рекурсивными вызовами и добавить базовый вариант:

In [ ]:
def fft(ys):
    N = len(ys)
    if N == 1:
        return ys

    He = fft(ys[::2])
    Ho = fft(ys[1::2])

    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)

    return np.tile(He, 2) + W * np.tile(Ho, 2)

И мы получаем те же результаты:

In [ ]:
hs4 = fft(ys)
np.sum(np.abs(hs - hs4))

1.6653345369377348e-16

Эта реализация FFT занимает время, пропорциональное $n \log n$. Она также занимает пространство, пропорциональное $n \log n$, и тратит некоторое время на создание и копирование массивов. Ее можно улучшить для работы «на месте»; в этом случае она не требует дополнительного пространства и тратит меньше времени на накладные расходы.